In [20]:
import tensorflow as tf

# from gensim.test.utils import common_texts,get_tmpfile
from gensim.models import word2vec
from gensim import corpora


from nltk.corpus import reuters,stopwords,brown
from nltk.tokenize import sent_tokenize,word_tokenize,RegexpTokenizer
# from nltk import Text

import numpy as np
import pandas as pd
import string
import re
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder

import sys

sys.path.append('..\\..\\NLP')
from word2vec_preprocess import *
# from pprint import pprint

In [267]:
text_ls = gen_text(2000)
sen_ls = gen_sentence(text_ls)

print(len(text_ls),len(sen_ls))

word_sen_ls = []
word_ls = []

for xx in sen_ls:
    word_sen_ls.append(gen_word(xx))
    word_ls = word_ls + gen_word(xx)


vocab = np.unique(word_ls)
print(len(word_ls),len(vocab))


2000 10240
242000 14340


In [293]:
word_ls = pd.Series(word_ls)
wc = word_ls.value_counts()
is_select = (wc>=20) * (wc<=5000)

# is_common_word = word_ls[is_select.values]
vocab = is_select[is_select==True].index.tolist()
print(len(vocab))

1461


In [294]:
wv_model = word2vec.Word2Vec(word_sen_ls,size=100,window=5)
model_vocab = list(wv_model.wv.vocab.keys())

df_word_vec = pd.DataFrame(index=vocab,columns=['vec_'+str(i) for i in range(100)])

for i,word in enumerate(df_word_vec.index[:]):
    df_word_vec.loc[word] = wv_model.wv.word_vec(word).round(4)


df_word_vec.head()

,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_90,vec_91,vec_92,vec_93,vec_94,vec_95,vec_96,vec_97,vec_98,vec_99
said,-0.3698,0.3696,0.3963,0.4458,-0.6512,-0.883,0.0827,1.3464,0.1074,0.1266,...,-0.8156,0.351,-1.0272,0.3028,-0.0771,-0.3193,-0.2752,-0.0613,-0.493,0.909
and,-0.145,0.4768,0.0581,0.062,-0.718,-0.1272,0.2745,0.2334,0.0191,0.2254,...,-0.4696,-0.2244,-0.4051,0.0807,0.5272,0.3785,-0.347,-0.0491,0.0785,0.2603
a,-0.3445,0.29,0.008,-0.0287,-0.9272,-0.4071,0.3112,0.0827,-0.0772,0.1188,...,0.0821,-0.1983,-0.1212,0.4562,0.171,0.2468,0.043,-0.3071,0.162,0.4359
mln,-0.2749,0.8201,-0.5405,-0.1533,-1.359,0.3779,0.5692,-0.5865,-1.1716,-0.1018,...,-0.348,-0.6756,-0.2584,0.5707,0.6061,1.1029,0.063,-0.5164,1.216,0.6045
for,-0.1593,0.4789,-0.1663,0.2015,-0.9665,-0.2836,0.3011,-0.0113,-0.2179,0.1938,...,-0.2255,-0.358,-0.2739,0.3162,0.2152,0.4208,-0.0376,-0.2174,0.4478,0.5019


In [295]:
df_word_vec.shape

(1461, 100)

In [296]:
wv_model.wv.most_similar('news')

[('reaffirmed', 0.9989835023880005),
 ('crisis', 0.9989787340164185),
 ('met', 0.9988820552825928),
 ('strength', 0.9987492561340332),
 ('expansion', 0.99871426820755),
 ('direction', 0.9986786842346191),
 ('governments', 0.9986090660095215),
 ('gulf', 0.9984222650527954),
 ('mexico', 0.9983996152877808),
 ('mideast', 0.9983780980110168)]

In [297]:
def gen_x_y(sentence):

    window = 4
    n = len(sentence)
    if n<=window:
        pass

    x_input = []
    y_input = []

    for k in range(n-4): #### 遍历句子长度 ####
        # for w in range(window): #### 对窗口内所有单词组合全部拿出来 #### 使用t~t+3单词预测t+4
        x_tmp,y_tmp = sentence[k:k+window],sentence[k+window]

        x_input.append(x_tmp)
        y_input.append(y_tmp)

        # print(x_tmp,y_tmp)

    x_input = pd.DataFrame(x_input)
    y_input = pd.DataFrame(y_input)

    return x_input,y_input

In [298]:
X = pd.DataFrame()
y = pd.DataFrame()

for sen in word_sen_ls[:]:

    sen = list(filter(lambda x:x in vocab,sen))

    X_tmp,y_tmp = gen_x_y(sen)

    X = pd.concat([X,X_tmp])
    y = pd.concat([y,y_tmp])

In [299]:
X.shape,y.shape

((127777, 4), (127777, 1))

In [300]:
len(np.unique(X)),len(np.unique(y))

(1461, 1461)

In [301]:
# X[~X.isin(vocab)]='the'
# y[~y.isin(vocab)]='the'

# y.isin(vocab).mean()

In [302]:
# X.iloc[0,0] = df_word_vec.loc['the'].values

In [303]:
X.shape

(127777, 4)

In [305]:
# pd.Series(X.values.ravel()).value_counts()

In [349]:
X.head()

,0,1,2,3
0,exporters,damage,from,trade
1,damage,from,trade,between
2,from,trade,between,us
3,trade,between,us,and
4,between,us,and,japan


In [353]:
y.head()

,0
0,between
1,us
2,and
3,japan
4,has


In [307]:
X_input = np.zeros(shape=[X.shape[0],X.shape[1],100])

In [308]:
X_input.shape

(127777, 4, 100)

In [309]:
X_input[0,0] = df_word_vec.loc[X.iloc[0,0]]

In [310]:
for i in range(X_input.shape[0]):

    if i%1000==0:
        print(i)

    for j in range(X_input.shape[1]):
        X_input[i,j] = df_word_vec.loc[X.iloc[i,j]]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000


In [311]:
# X_input = X.values[:,:,np.newaxis]

In [312]:
X_input.shape

(127777, 4, 100)

In [313]:
# y.head()

In [314]:
X.shape

(127777, 4)

In [315]:
ohe = OneHotEncoder()
# ohe = OrdinalEncoder()
ohe.fit(np.array(vocab).reshape(-1,1))

# X_trans = X.apply(lambda x:np.squeeze(ohe.transform(x.reshape(-1,1)),axis=1))
# y_trans = y.apply(lambda x:np.squeeze(ohe.transform(y[0].reshape(-1,1))))
y_trans = ohe.transform(y).toarray()

In [343]:
np.argmax(ohe.transform(np.array([['and']])).toarray())

72

In [ ]:
ohe.transform

In [319]:
#### RNN CELL ####
rnn_mod = 1
num_unit = 128

if rnn_mod==1:
    cell = tf.nn.rnn_cell.BasicRNNCell(num_units=num_unit)
if rnn_mod==2:
    cell = tf.nn.rnn_cell.LSTMCell(num_units=num_unit)
if rnn_mod==3:
    cell = tf.nn.rnn_cell.GRUCell(num_units=num_unit)


In [320]:
input_x = tf.placeholder(tf.float32,[None,4,100])
input_y = tf.placeholder(tf.float32,[None,len(vocab)])

In [321]:
# embed_size = 100

# word_embeddings = tf.Variable(tf.truncated_normal([vob_size,embed_size],stddev=1,mean=0.1))
# embedding_word_ids = tf.nn.embedding_lookup(word_embeddings,input_x)

# weights = tf.Variable(tf.truncated_normal([vob_size,embed_size],mean=1,stddev=1))
# # bias = tf.Variable(tf.truncated_normal([vob_size]))


In [322]:

# tf.reset_default_graph()
#create a BasicRNNCell

with tf.variable_scope('rnn1',reuse=tf.AUTO_REUSE):

    rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=128)
    # rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=128)
    # rnn_cell = tf.nn.rnn_cell.GRUCell(num_units=128)

    #'outputs' is a tensor of shape [batch_size, max_time, cell_state_size]

    #defining initial state
    initial_state = tf.placeholder(tf.float32,[None,128])

    #'state' is a tensor of shape [batch_size, cell_state_size]
    outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell,inputs=input_x,dtype=tf.float32)

    print(outputs, state)


Tensor("rnn1_7/rnn/transpose_1:0", shape=(?, 4, 128), dtype=float32) Tensor("rnn1_7/rnn/while/Exit_3:0", shape=(?, 128), dtype=float32)


In [327]:
# input_y.

In [355]:
# state_drop = tf.layers.dropout(0.2)

with tf.variable_scope('logit2',reuse=tf.AUTO_REUSE):
    logits = tf.layers.dense(state,units=3000)
    logits2 = tf.layers.dense(state,units=len(vocab))

    error = tf.subtract(input_y,logits2)
    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels=input_y,logits=logits2))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(loss)


tf.summary.scalar('loss',loss)
summ_op = tf.summary.merge_all()

In [330]:
optimizer

<tf.Operation 'logit2/Adam' type=NoOp>

In [331]:
# for embedding loss
# loss = tf.reduce_sum(
#     tf.nn.sampled_softmax_loss(weights=weights,
#                                 biases = bias,
#                                 labels = input_y,
#                                 inputs = embedding_word_ids,
#                                 num_sampled = 20,
#                                 num_classes = vob_size
#                                 ))



In [365]:
train_log_path = "C:/Users/qqmai/Desktop/study/deep learning/tmp/train_tmp"

with tf.Session() as sess:

    init=tf.global_variables_initializer()
    sess.run([init])

    summary_writer_tr = tf.summary.FileWriter(train_log_path, graph=sess.graph)

    batch_size = 1000

    for ii in range(1000):
        i = ii%100
        _,loss_tmp,smm = sess.run([optimizer,loss,summ_op],
                            feed_dict={input_x:X_input[batch_size*i:batch_size*(i+1)],
                                      input_y:y_trans[batch_size*i:batch_size*(i+1)]})

        summary_writer_tr.add_summary(smm,ii)


        if i%10==0:
            print(i,loss_tmp)

        
    y_pred = sess.run(logits2,feed_dict=({input_x:X_input}))

summary_writer_tr.close()
sess.close()

0 7295.3413
10 7207.38
20 7141.4883
30 6987.3774
40 6831.203
50 6723.9097
60 6530.462
70 6412.95
80 6315.078
90 6319.9604
0 6381.051
10 6063.3994
20 6321.379
30 6188.9287
40 6150.1025
50 6295.8955
60 6161.251
70 6185.363
80 6133.0503
90 6214.706
0 6344.081
10 5904.896
20 6245.7637
30 6054.5986
40 6035.6436
50 6233.875
60 6061.6616
70 6093.677
80 6036.61
90 6158.4307
0 6301.8184
10 5784.8667
20 6185.036
30 5941.638
40 5941.464
50 6184.0166
60 5987.222
70 6020.789
80 5962.2305
90 6117.17
0 6268.655
10 5711.8374
20 6146.4985
30 5870.209
40 5879.628
50 6148.0425
60 5937.5967
70 5965.798
80 5908.637
90 6082.5747
0 6240.6416
10 5660.191
20 6115.365
30 5815.7275
40 5831.6675
50 6115.114
60 5897.397
70 5921.334
80 5865.314
90 6052.4717
0 6216.021
10 5618.3945
20 6086.9004
30 5771.588
40 5790.9077
50 6085.4316
60 5863.4097
70 5884.6406
80 5826.5977
90 6026.3594
0 6194.132
10 5584.2256
20 6060.8203
30 5733.3384
40 5754.6494
50 6058.5303
60 5832.9707
70 5852.1816
80 5791.455
90 6003.2236
0 6173.8

In [362]:
# 22//10

2

In [348]:
np.apply_along_axis(lambda x:np.argmax(x),1,y_pred[:10])

array([1393, 1432,   72,   72,   72,  504,   72,   72,   72,   72],
      dtype=int64)

In [339]:
y_pred[:100][:,72]

array([3.3258963, 3.3662004, 3.3701138, 3.3707998, 3.3550406, 3.3436682,
       3.3490684, 3.3465526, 3.3631418, 3.3563485, 3.3418279, 3.3773727,
       3.375233 , 3.3444042, 3.3597507, 3.3397362, 3.3618507, 3.3771994,
       3.3802953, 3.3570595, 3.366798 , 3.3493881, 3.307503 , 3.320223 ,
       3.312397 , 3.318333 , 3.3782504, 3.3586054, 3.3645713, 3.3790708,
       3.3254762, 3.3486302, 3.3873694, 3.3716464, 3.3900445, 3.342552 ,
       3.2104769, 3.2315495, 3.3513896, 3.4010973, 3.3164034, 3.3858533,
       3.326925 , 3.2544281, 3.156462 , 3.227486 , 3.372555 , 3.3729856,
       3.3626654, 3.32598  , 3.3576622, 3.3706458, 3.3052306, 3.2808628,
       3.317006 , 3.3447092, 3.2765868, 3.3626883, 3.340518 , 3.3262165,
       3.3678746, 3.3806834, 3.3778763, 3.3818972, 3.3794982, 3.3529086,
       3.3382385, 3.336514 , 3.3588762, 3.2737567, 3.1937103, 3.2642863,
       3.2994494, 3.356535 , 3.315668 , 3.3406982, 3.3100796, 3.27799  ,
       3.342139 , 3.4026654, 3.3705902, 3.3323433, 

In [92]:
X

(1154, 1)

In [105]:
# X_trans.values[:,:,np.newaxis].shape

In [161]:
vector2 = np.dot(wgs,vector1)
order2 = vector2.argsort()[::-1]

In [162]:
ohe.inverse_transform(order2[:10,np.newaxis])

array([['export'],
       ['government'],
       ['said'],
       ['company'],
       ['likely'],
       ['expected'],
       ['market'],
       ['exchange'],
       ['spokesman'],
       ['last']], dtype='<U19')

In [77]:
1 * [1,2,3]

[1, 2, 3]